# Automatic Data Type Discovery

In [1]:
from scipy.io import savemat
import openml as oml
import pandas as pd
import numpy as np
import random

## 1. Heuristic Methods

In [2]:
def infer_feature_type(feature):
    """
    Infer data types for the given feature using simple logic
    Possible data types to infer: boolean, date, float, integer, string
    Feature that is not either a boolean, a date, a float or an integer, is classified as a string
    """
    data_type = ""
    types = ["date","float64","int64","string"]
    weights = [0,0,0,0] #Weights corresponding to the data types
    feature_len = len(feature)
    
    indices_number = int(0.1 * feature_len) #Number of different values to check in a feature
    indices = random.sample(range(0,feature_len), min(indices_number,feature_len)) #Array of random indices
    
    # If the feature only contains two different unique values, then infer it as boolean
    if len(pd.unique(feature)) == 2:
        data_type = "bool"
    else:
        for i in indices:
            try:
                if (len(feature[i]) <= 10 
                    and (((feature[i][2:3]=='-' or feature[i][2:3]=='/') 
                    and (feature[i][5:6]=='-' or feature[i][5:6]=='/')) 
                    or ((feature[i][4:5]=='-' or feature[i][4:5]=='/')
                    and (feature[i][7:8]=='-' or feature[i][7:8]=='/')))):
                    weights[0] += 1 #Date
                else:
                    weights[3] += 1 #String
            except (TypeError,ValueError,IndexError):
                try:
                    int(feature[i]) # numeric
                    if ('.' in str(feature[i])):
                        if isinstance(feature[i], np.float64):
                            if feature[i].is_integer():
                                weights[2] += 1 # Integer
                            else:
                                weights[1] += 1 # Float
                        else:
                            weights[1] += 1 # Float
                    else:
                        weights[2] += 1 #Integer
                except (TypeError,ValueError,IndexError):
                    weights[3] += 1 #String
#         print ("Date: {}, Float64: {}, Int64: {}, String: {}".format(weights[0],weights[1],weights[2],weights[3])) #For debugging purposes
        data_type = types[weights.index(max(weights))]
        
    return data_type

def discover_type_heuristic(data):
    """
    Automatically infer data types for each feature in the dataframe using simple logic
    ---
    input:

    data : numpy array or dataframe (numeric data needs to be 64 bit)
        
    """
#     df = pd.DataFrame(data)
#     print(df)
    result = []
    if isinstance(data, np.ndarray):
        # convert float32 to float64
        data = np.array(data, dtype='float64')
        df = pd.DataFrame(data)
    else:
        df = data

    for column in df.columns:        
        #print ("Trying to automatically infer the data type of the",column,"feature...") #For debugging purposes
        type_inferred = infer_feature_type(df[column])
        result.append(type_inferred)
        #print ("Result:",inferredType) #For debugging purposes
    return result

### 1.1 Get data from csv

In [3]:
df = pd.read_csv("datasets_csv/tips.csv")
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
discover_type_heuristic(df)

['float64', 'float64', 'bool', 'bool', 'string', 'bool', 'int64']

### 1.2 Get data from openml

In [5]:
data = oml.datasets.get_dataset(1480)
Xy = data.get_data()
Xy.shape

(583, 11)

In [6]:
discover_type_heuristic(Xy)

['int64',
 'bool',
 'float64',
 'float64',
 'int64',
 'int64',
 'int64',
 'float64',
 'float64',
 'float64',
 'bool']

## 2. Bayesian Method
To apply bayesian methos, dataset needs to be parsed into tabular numeric form beforehand.

### Get data from openml

In [28]:
data = oml.datasets.get_dataset(40498)
Xy = data.get_data()
Xy.shape

(4898, 12)

### Generate mat file

In [62]:
def generate_mat(data, extra_cardinality=100):
    simple_types = discover_type_heuristic(data)
    # map simple types to meta types
    # 1: real (w positive: all real | positive | interval)
    # 2: real (w/o positive: all real | interval)
    # 3: binary data
    # 4: discrete (non-binary: categorical | ordinal | count)
    # note: for now, the implemented bayesian method version by isabel can distinguish between real, postive real, categorical and count
    # the binary data should be mapped to meta type 4 discrete instead of meta type 3 due to the limited implemented version. This may change
    # if the extended version has been implemented by isabel.
    meta_types = [] 
    for i in range(len(simple_types)):
#         print(simple_types[i])
        if simple_types[i] == "bool":
            meta_types.append(4) # may change in the future
        elif simple_types[i] == "int64" or simple_types[i] == "float64":
            if (len(np.unique(data[:,i])) < 0.02 * len(data[:,i]) and len(np.unique(data[:,i])) < 50):
                meta_types.append(4)
            else:
                if (data[:,i] > 0).all():
                    meta_types.append(1)
                else:
                    meta_types.append(2)
    discrete_cardinality = [] # number of unique for discrete feature, 1 for others
    for i in range(len(meta_types)):
        if (meta_types[i] == 4):
            discrete_cardinality.append(len(np.unique(data[:,i])) + extra_cardinality) # +100 to be on the safe side, otherwise one hot bug
        else:
            discrete_cardinality.append(1)
    data_dict = {'X': data,
                 'T': np.asarray(meta_types),
                 'R': np.asarray(discrete_cardinality)}
    print(data_dict)
    savemat('bayesian/data/temp/data.mat', data_dict, oned_as='row')

In [30]:
generate_mat(Xy)

{'X': array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  3.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  3.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  3.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  3.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  4.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  3.  ]], dtype=float32), 'T': array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 4]), 'R': array([  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1, 107])}


### Call bayesian model

In [31]:
from bayesian.bin import abda

In [32]:
weights = abda.main(seed=1337, dataset='bayesian/data/temp/data.mat', exp_id=None, args_output='./exp/temp/', args_miss=None, verbose=1,
         args_col_split_threshold=0.8, args_min_inst_slice=500, args_leaf_type='pm',
         args_type_param_map='spicky-prior-1', args_param_init='default', args_param_weight_init='uniform',
         args_n_iters=1, args_burn_in=4000, args_w_unif_prior=100, args_save_samples=1,
         args_ll_history=1, args_omega_prior='uniform', args_plot_iter=10, args_omega_unif_prior=10,
         args_leaf_omega_unif_prior=0.1, args_cat_unif_prior=1)


Loaded bayesian/data/temp/data.mat
	with shape: (4898, 12)
	meta-types:[1 1 2 1 1 1 1 1 1 1 1 4]
	maximal discrete cardinality: [  1   1   1   1   1   1   1   1   1   1   1 107]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([ 3.79999995, 14.19999981]), array([0.08      , 1.10000002]), array([0.        , 1.65999997]), array([ 0.60000002, 65.80000305]), array([0.009     , 0.34599999]), array([  2., 289.]), array([  9., 440.]), array([0.98711002, 1.03898001]), array([2.72000003, 3.81999993]), array([0.22      , 1.08000004]), array([ 8.        , 14.19999981]), array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
 

       4.50247634e-11, 4.50247634e-11, 4.50247634e-11])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC1BE2CC0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC1BE2B38>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1BE2DA0>}
gamma fit params (59.216345594122124, 0, 0.11373646650669383)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3811748>}
{'params': {'alpha': 59.216345594122124, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC38113C8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3811A90>}
gamma fit params (8.225815019050623, 0, 0.03333915098439663)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC052E198>}
gamma fit params (1037314.7824667657, 0, 9.592935551802894e-07)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC052E710>}
{'params': {'alpha': 1037314.7824667657, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC052E9B0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC052EB38>}
gamma fit params (20.19787985173812, 0, 0.6849826542372451)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC052E9E8>}
{'params': {'alpha': 20.19787985173812, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC052EBE0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC052E550>}
g

 3.73552483e-11 3.73552483e-11 3.73552483e-11])) + 0.3333333333333333*(Geometric(V11|p=0.5)) + 0.3333333333333333*(Poisson(V11|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Done iteration 1/1 in 0.21232744225699207


------------------
Done in 0.2222792425420721
Dumped id to ./exp/temp/data.mat\20181029-143857\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-143857\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-143857


In [33]:
weights

{0: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.001975280707022,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.998024719292978},
 1: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.2505448190105733,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.7494551809894267},
 2: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.17809000957006083,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.821909990429939},
 3: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.2148417757517452,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.7851582242482545},
 4: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.1705409631051393,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.8294590368948606},
 5: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.0013720122627796153,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.9986279877372204},
 6: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.000991269639289043,
  <Type.POSITIVE: (3, <MetaType.REAL: 1>)>: 0.9990087303607109},
 7: {<Type.REAL: (1, <MetaType.REAL: 1>)>: 0.4025493630293431,
  <Type.POSITIVE: (3, <MetaType.REAL: 

## 3. Evaluation
We evaluate the bayesian method on the 30 most runs openml datasets. The ids of the 30 openml datasets are as follows: <br>
31, 1464, 334, 50, 333, 1494, 3, 1510, 1489, 37, 1479, 1487, 1063, 1471, 1467, 1480, 1068, 1492, 1050, 1462, 1046, 1049, 335, 

In [34]:
def get_statistical_type(weights):
    """
    Retrieve the key with the higher value from 'weights'
    The retrieved key is the statisical type of the corresponding feature
    """
    statistical_types = []
    for i in range(len(weights)):
#         print(max(weights[i], key=weights[i].get))
        statistical_types.append(str(max(weights[i], key=weights[i].get)))
    return statistical_types

In [35]:
def statistical_type_accurracy(ground_truth, statistical_types):
    count = 0 # number of correct predictions
    for i in range(len(ground_truth)):
        if ground_truth[i] == statistical_types[i]:
            count += 1
    return count / len(ground_truth)

In [67]:
def evaluate_bayesian(dataset_id, ground_truth):
    data = oml.datasets.get_dataset(dataset_id)
    Xy = data.get_data()
    Xy.shape
    try:
        generate_mat(Xy)
        weights = abda.main(seed=1337, dataset='bayesian/data/temp/data.mat', exp_id=None, args_output='./exp/temp/', args_miss=None, verbose=1,
         args_col_split_threshold=0.8, args_min_inst_slice=500, args_leaf_type='pm',
         args_type_param_map='spicky-prior-1', args_param_init='default', args_param_weight_init='uniform',
         args_n_iters=1, args_burn_in=4000, args_w_unif_prior=100, args_save_samples=1,
         args_ll_history=1, args_omega_prior='uniform', args_plot_iter=10, args_omega_unif_prior=10,
         args_leaf_omega_unif_prior=0.1, args_cat_unif_prior=1)
    except:
        generate_mat(Xy, 200)
        weights = abda.main(seed=1337, dataset='bayesian/data/temp/data.mat', exp_id=None, args_output='./exp/temp/', args_miss=None, verbose=1,
         args_col_split_threshold=0.8, args_min_inst_slice=500, args_leaf_type='pm',
         args_type_param_map='spicky-prior-1', args_param_init='default', args_param_weight_init='uniform',
         args_n_iters=1, args_burn_in=4000, args_w_unif_prior=100, args_save_samples=1,
         args_ll_history=1, args_omega_prior='uniform', args_plot_iter=10, args_omega_unif_prior=10,
         args_leaf_omega_unif_prior=0.1, args_cat_unif_prior=1)
    statistical_types = get_statistical_type(weights)
    print(statistical_types)
    accuracy = statistical_type_accurracy(ground_truth, statistical_types)
    return accuracy

In [37]:
# 31
ground_truth = ['Type.COUNT', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.POSITIVE',
                'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.CATEGORICAL',
                'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.CATEGORICAL',
                'Type.CATEGORICAL', 'Type.CATEGOTICAL', 'Type.COUNT', 'Type.CATEGORICAL',
                'Type.CATEGORICAL', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(31, ground_truth)

{'X': array([[ 0.,  6.,  4., ...,  1.,  0.,  0.],
       [ 1., 48.,  2., ...,  0.,  0.,  1.],
       [ 3., 12.,  4., ...,  0.,  0.,  0.],
       ...,
       [ 3., 12.,  2., ...,  0.,  0.,  0.],
       [ 0., 45.,  2., ...,  1.,  0.,  1.],
       [ 1., 45.,  4., ...,  0.,  0.,  0.]], dtype=float32), 'T': array([4, 1, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4]), 'R': array([104,   1, 105, 110,   1, 105, 105, 104, 104, 103, 104, 104,   1,
       103, 103, 104, 104, 102, 102, 102, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (1000, 21)
	meta-types:[4 1 4 4 1 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4]
	maximal discrete cardinality: [104   1 105 110   1 105 105 104 104 103 104 104   1 103 103 104 104 102
 102 102 102]
	meta types [<MetaType.DISCRETE: 3> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <Me

        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101], dtype=int64)]
	translated discrete features (starting from 0)
MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gaussian'>, {'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EB91219E8>})])), (<Type.POSITIVE: (3, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gamma'>, {'params': {'alpha': None, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EB9121A20>}), (<class 'bayesian.spn.structure.leaves.parametric.Parametric.Exponential'>, {'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EBC87FF98>})]))])
MT MetaType.DISCRETE OrderedDict([(<Type.CATEGORICAL: (4, <MetaType.DISCRETE: 3>)>, {<class 'bayesian.spn.structure.leaves.parametric.P

       2.04498973e-10, 2.04498973e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC40D0438>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC40D03C8>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC40D0710>}
gamma fit params (2.6899102434090474, 0, 1770.491868399016)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3811550>}
{'params': {'alpha': 2.6899102434090474, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3811C88>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3811518>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC0A8AB38>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0A8A128>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([6.33946819e-02, 2.65848665e-01, 6.09406940e-01, 6.13496922e-02,
       2.04498973e-10, 2.04498973e-10, 2.04498973e-10, 2.04498973e-10,
       2.04498973e-10, 2.04498973e-10, 2.04498973e-10, 2.04498973e-10,
       2.04498973e-10, 2.04498973e-10, 2.04498973e-10, 2.04498973e-10,
       2.04498973e-10, 2.04498973e-10, 2.04498973e-10, 2.04498973e-10,
       2.04498973e-10, 2.04498973e-10, 2.04498973e-10, 2.04498973e-1

       2.04498973e-10, 2.04498973e-10, 2.04498973e-10, 2.04498973e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC20676A0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC2067E80>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC20679B0>}
gamma fit params (12.07173774082905, 0, 2.926099716777992)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC052E5C0>}
{'params': {'alpha': 12.07173774082905, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC052E358>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC5A10B70>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0

...

       1.95694712e-10, 1.95694712e-10, 1.95694712e-10, 1.95694712e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B06E10>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B06EB8>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B06F28>}
gamma fit params (10.002108141330806, 0, 3.575175852304238)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B16048>}
{'params': {'alpha': 10.002108141330806, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B160F0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B16198>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.

{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B16908>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B169B0>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([8.43444210e-01, 1.56555770e-01, 1.95694712e-10, 1.95694712e-10,
       1.95694712e-10, 1.95694712e-10, 1.95694712e-10, 1.95694712e-10,
       1.95694712e-10, 1.95694712e-10, 1.95694712e-10, 1.95694712e-10,
       1.95694712e-10, 1.95694712e-10, 1.95694712e-10, 1.95694712e-10,
       1.95694712e-10, 1.95694712e-10, 1.95694712e-10, 1.95694712e-10,
       1.95694712e-10, 1.95694712e-10, 1.95694712e-10, 1.95694712e-10,
   

       1.95694712e-10, 1.95694712e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B0A358>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B0A630>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B0A6A0>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([2.68096507e-10, 9.99999973e-01, 2.68096507e-10, 2.68096507e-10,
       2.68096507e-10, 2.68096507e-10, 2.68096507e-10, 2.68096507e-10,
       2.68096507e-10, 2.68096507e-10, 2.68096507e-10, 2.68096507e-10,
       2.68096507e-10, 2.68096507e-10, 2.68096507e-10, 2.68096507e-10,
       2.

 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([7.31884004e-01, 7.24637627e-10, 2.68115923e-01, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10, 7.24637627e-10, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10, 7.24637627e-10, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10, 7.24637627e-10, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10, 7.24637627e-10, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10, 7.24637627e-10, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10, 7.24637627e-10, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10, 7.24637627e-10, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10, 7.24637627e-10, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10, 7.24637627e-10, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10, 7.24637627e-10, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10, 7.24637627e-10, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10, 7.24637627e-10, 7.24637627e-10,
       7.24637627e-10, 7.24637627e-10

 1.95694712e-10 1.95694712e-10])) + 0.3333333333333333*(Geometric(V20|p=0.5)) + 0.3333333333333333*(Poisson(V20|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Done iteration 1/1 in 0.1340149454645143


------------------
Done in 0.14063618886711993
Dumped id to ./exp/temp/data.mat\20181029-143921\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-143921\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-143921
['Type.COUNT', 'Type.POSITIVE', 'Type.COUNT', 'Type.COUNT', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.COUNT', 'Type.COUNT', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL']


In [38]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.5714285714285714


In [50]:
# 1464
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1464, ground_truth)

{'X': array([[2.00e+00, 5.00e+01, 1.25e+04, 9.80e+01, 1.00e+00],
       [0.00e+00, 1.30e+01, 3.25e+03, 2.80e+01, 1.00e+00],
       [1.00e+00, 1.60e+01, 4.00e+03, 3.50e+01, 1.00e+00],
       ...,
       [2.30e+01, 3.00e+00, 7.50e+02, 6.20e+01, 0.00e+00],
       [3.90e+01, 1.00e+00, 2.50e+02, 3.90e+01, 0.00e+00],
       [7.20e+01, 1.00e+00, 2.50e+02, 7.20e+01, 0.00e+00]], dtype=float32), 'T': array([2, 1, 1, 1, 4]), 'R': array([  1,   1,   1,   1, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (748, 5)
	meta-types:[2 1 1 1 4]
	maximal discrete cardinality: [  1   1   1   1 102]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([ 0., 74.]), array([ 1., 50.]), array([  250., 12500.]), array([ 2., 98.]), array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  

{'params': {'alpha': 3.241626047198757, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC228D908>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC228DB70>}
gamma fit params (5.215812466747469, 0, 9.758734487060758)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC228DC88>}
{'params': {'alpha': 5.215812466747469, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC228D668>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC228DA58>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

Done iteration 1/1 in 0.03844845077981063


------------------
Done in 0.041978359472750526
Dumped id to ./exp/temp/data.mat\20181029-144446\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-144446\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-144446
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']


In [51]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [39]:
# 334
ground_truth = ['Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(334, ground_truth)

{'X': array([[0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 3., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 2., 2., ..., 2., 2., 1.],
       [0., 2., 2., ..., 2., 3., 0.],
       [0., 2., 2., ..., 2., 3., 1.]], dtype=float32), 'T': array([4, 4, 4, 4, 4, 4, 4]), 'R': array([102, 103, 103, 102, 103, 104, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (601, 7)
	meta-types:[4 4 4 4 4 4 4]
	maximal discrete cardinality: [102 103 103 102 103 104 102]
	meta types [<MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3>]
	domains [array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,

       3.32225902e-10, 3.32225902e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3811C88>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC1BE2BE0>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1BE2DA0>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([9.99999966e-01, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.333

       3.32225902e-10, 3.32225902e-10, 3.32225902e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC0A8A0F0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC0A8AE48>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0A8A128>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([5.01661113e-01, 4.98338854e-01, 3.32225902e-10, 3.32225902e-10,
       3.32225902e-10, 3.32225902e-10, 3.32225902e-10, 3.32225902e-10,
       3.32225902e-10, 3.32225902e-10, 3.32225902e-10, 3.32225902e-10,
       3.32225902e-10, 3.32225902e-10, 3.32225902e-10, 3.32225902e-

 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([6.79999977e-01, 3.19999989e-01, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3

       3.33333322e-10, 3.33333322e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B2E7B8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B2EDD8>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B2E710>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([3.33333322e-01, 3.36666655e-01, 3.29999989e-01, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.33333322e-10, 3.33333322e-10, 3.33333322e-10, 3.33333322e-10,
       3.

 3.33333322e-10 3.33333322e-10 3.33333322e-10 3.33333322e-10])) + 0.3333333333333333*(Geometric(V5|p=0.5)) + 0.3333333333333333*(Poisson(V5|mean=2))))))
[0, 1, 2, 3, 4, 5, 6]
Done iteration 1/1 in 0.035241452544482854


------------------
Done in 0.08687672952498815
Dumped id to ./exp/temp/data.mat\20181029-143946\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-143946\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-143946
['Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL']


In [40]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [53]:
# 50
ground_truth = ['Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL',
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(50, ground_truth)

{'X': array([[2., 2., 2., ..., 1., 1., 1.],
       [2., 2., 2., ..., 2., 1., 1.],
       [2., 2., 2., ..., 1., 2., 1.],
       ...,
       [1., 2., 1., ..., 1., 2., 0.],
       [1., 2., 1., ..., 1., 2., 0.],
       [1., 1., 2., ..., 2., 2., 0.]], dtype=float32), 'T': array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4]), 'R': array([103, 103, 103, 103, 103, 103, 103, 103, 103, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (958, 10)
	meta-types:[4 4 4 4 4 4 4 4 4 4]
	maximal discrete cardinality: [103 103 103 103 103 103 103 103 103 102]
	meta types [<MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3>]
	domains [array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36, 

MT MetaType.DISCRETE OrderedDict([(<Type.CATEGORICAL: (4, <MetaType.DISCRETE: 3>)>, {<class 'bayesian.spn.structure.leaves.parametric.Parametric.Categorical'>: {'params': {'p': None}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EBFD78748>}}), (<Type.COUNT: (6, <MetaType.DISCRETE: 3>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Geometric'>, {'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC091E128>}), (<class 'bayesian.spn.structure.leaves.parametric.Parametric.Poisson'>, {'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC20BDF28>})]))])




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([2.34939752e-01, 3.04216858e-01, 4.60843359e-01, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10

       3.01204810e-10, 3.01204810e-10, 3.01204810e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B17EF0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B17AC8>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B17D30>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([2.34939752e-01, 3.04216858e-01, 4.60843359e-01, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.0120481

 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([1.89759031e-01, 4.39759023e-01, 3.70481917e-01, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10, 3.01204810e-10, 3.01204810e-10,
       3.01204810e-10, 3.01204810e-10

       1.59744406e-10, 1.59744406e-10, 1.59744406e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC228D668>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC228D630>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC228DB00>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([2.74760379e-01, 3.65814691e-01, 3.59424914e-01, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.5974440

 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([2.26837057e-01, 3.01916928e-01, 4.71245999e-01, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10, 1.59744406e-10, 1.59744406e-10,
       1.59744406e-10, 1.59744406e-10

 1.59744406e-10 1.59744406e-10 1.59744406e-10])) + 0.3333333333333333*(Geometric(V8|p=0.5)) + 0.3333333333333333*(Poisson(V8|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Done iteration 1/1 in 0.0568215821033391


------------------
Done in 0.06023661678273129
Dumped id to ./exp/temp/data.mat\20181029-144552\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-144552\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-144552
['Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT']


In [54]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.0


In [55]:
# 333
ground_truth = ['Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(333, ground_truth)

{'X': array([[1., 0., 0., ..., 0., 2., 0.],
       [1., 0., 0., ..., 0., 2., 1.],
       [1., 0., 0., ..., 2., 1., 0.],
       ...,
       [1., 2., 2., ..., 2., 2., 1.],
       [1., 2., 2., ..., 2., 3., 0.],
       [1., 2., 2., ..., 2., 3., 1.]], dtype=float32), 'T': array([4, 4, 4, 4, 4, 4, 4]), 'R': array([102, 103, 103, 102, 103, 104, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (556, 7)
	meta-types:[4 4 4 4 4 4 4]
	maximal discrete cardinality: [102 103 103 102 103 104 102]
	meta types [<MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3>
 <MetaType.DISCRETE: 3>]
	domains [array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,

       3.67647045e-10, 3.67647045e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC052ECC0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC052E898>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC052E3C8>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([3.52112663e-10, 9.99999964e-01, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.521

       3.67647045e-10, 3.67647045e-10, 3.67647045e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B17438>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B17C50>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B17400>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([5.11029393e-01, 4.88970570e-01, 3.67647045e-10, 3.67647045e-10,
       3.67647045e-10, 3.67647045e-10, 3.67647045e-10, 3.67647045e-10,
       3.67647045e-10, 3.67647045e-10, 3.67647045e-10, 3.67647045e-10,
       3.67647045e-10, 3.67647045e-10, 3.67647045e-10, 3.67647045e-

 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([5.03521109e-01, 4.96478856e-01, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3

       3.52112663e-10, 3.52112663e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B177B8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B17278>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B17668>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([3.27464777e-01, 3.27464777e-01, 3.45070410e-01, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.52112663e-10, 3.52112663e-10, 3.52112663e-10, 3.52112663e-10,
       3.

 3.52112663e-10 3.52112663e-10 3.52112663e-10 3.52112663e-10])) + 0.3333333333333333*(Geometric(V5|p=0.5)) + 0.3333333333333333*(Poisson(V5|mean=2))))))
[0, 1, 2, 3, 4, 5, 6]
Done iteration 1/1 in 0.03143442509713168


------------------
Done in 0.05335286059971622
Dumped id to ./exp/temp/data.mat\20181029-144941\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-144941\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-144941
['Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL']


In [56]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [69]:
# 1494
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.COUNT', 'Type.POSITIVE', 'Type.CATEGORICAL', 
                'Type.COUNT', 'Type.COUNT', 'Type.POSITIVE', 'Type.COUNT', 'Type.COUNT', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.COUNT', 'Type.COUNT', 
                'Type.COUNT', 'Type.POSITIVE', 'Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.REAL', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.COUNT', 'Type.POSITIVE', 'Type.COUNT', 
                'Type.COUNTL', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1494, ground_truth)

{'X': array([[3.919 , 2.6909, 0.    , ..., 0.    , 0.    , 1.    ],
       [4.17  , 2.1144, 0.    , ..., 0.    , 0.    , 1.    ],
       [3.932 , 3.2512, 0.    , ..., 0.    , 0.    , 1.    ],
       ...,
       [4.869 , 1.767 , 0.    , ..., 1.    , 0.    , 0.    ],
       [5.158 , 1.6914, 2.    , ..., 0.    , 1.    , 0.    ],
       [5.076 , 2.6588, 2.    , ..., 0.    , 2.    , 0.    ]],
      dtype=float32), 'T': array([1, 1, 4, 4, 4, 4, 4, 2, 4, 4, 4, 2, 1, 2, 1, 2, 1, 1, 4, 4, 4, 1,
       4, 4, 4, 4, 1, 2, 4, 2, 1, 4, 4, 4, 4, 1, 1, 4, 1, 4, 4, 4]), 'R': array([  1,   1, 111, 104, 116, 113, 115,   1, 115, 112, 121,   1,   1,
         1,   1,   1,   1,   1, 103, 104, 104,   1, 113, 102, 102, 104,
         1,   1, 102,   1,   1, 108, 111, 116, 108,   1,   1, 108,   1,
       105, 117, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (1055, 42)
	meta-types:[1 1 4 4 4 4 4 2 4 4 4 2 1 2 1 2 1 1 4 4 4 1 4 4 4 4 1 2 4 2 1 4 4 4 4 1 1
 4 1 4 4 4]
	maximal discrete cardinality: [  1 

        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101], dtype=int64)]
	translated discrete features (starting from 0)
MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gaussian'>, {'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EB91219E8>})])), (<Type.POSITIVE: (3, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gamma'>, {'params': {'alpha': None, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EB9121A20>}), (<class 'bayesian.spn.structure.leaves.parametric.Parametric.Exponential'>, {'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EBC87FF98>})]))])
MT MetaType.DISCRETE OrderedDict([(<Type.CATEGORICAL: (4, <MetaType.DISCRETE: 3>)>, {<class 'bayesian.spn.structure.leaves.parametric.P

       195, 196, 197, 198, 199, 200, 201], dtype=int64)]
	translated discrete features (starting from 0)
MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gaussian'>, {'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EB91219E8>})])), (<Type.POSITIVE: (3, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gamma'>, {'params': {'alpha': None, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EB9121A20>}), (<class 'bayesian.spn.structure.leaves.parametric.Parametric.Exponential'>, {'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EBC87FF98>})]))])
MT MetaType.DISCRETE OrderedDict([(<Type.CATEGORICAL: (4, <MetaType.DISCRETE: 3>)>, {<class 'bayesian.spn.structure.leaves.parametric.Parametric.Categorica

       2.16450207e-10, 2.16450207e-10, 2.16450207e-10, 2.16450207e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B8FA20>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B8FC18>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B8F128>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.

       1.68634058e-10, 1.68634058e-10, 1.68634058e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3BA7CC0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3BA7E48>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3BA7CF8>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B7AD68>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B7A6A0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B7AF28>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B7AC88>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B7AC50>}
{

...

       3.53356864e-10, 3.53356864e-10, 3.53356864e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B73240>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B73400>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B737B8>}
gamma fit params (15.97829123371848, 0, 0.29418713895150755)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B73668>}
{'params': {'alpha': 15.97829123371848, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B73630>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B73978>}
gamma fit params (65.87032479174327, 0, 0.15052972239765272)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC

       3.22580623e-10, 3.22580623e-10, 3.22580623e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B73D30>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B73A58>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B73E48>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 

       3.22580624e-10, 3.22580624e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B84160>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B84048>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B84208>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

       3.22580623e-10, 3.22580623e-10, 3.22580623e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B84518>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B84470>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B84588>}
gamma fit params (474.9104866657212, 0, 0.008002818819209407)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B845F8>}
{'params': {'alpha': 474.9104866657212, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B845C0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B84748>}
gamma fit params (1328.6516716689125, 0, 0.006456038098831414)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000

 1.68634058e-10 1.68634058e-10])) + 0.3333333333333333*(Geometric(V41|p=0.5)) + 0.3333333333333333*(Poisson(V41|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
Done iteration 1/1 in 0.34162584231080473


------------------
Done in 0.35327390938755343
Dumped id to ./exp/temp/data.mat\20181029-151759\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-151759\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-151759
['Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.COUNT', 'Type.CATEGORICAL',

In [70]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.5476190476190477


In [72]:
# 3
ground_truth = ['Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL',
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL',
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 
                'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 'Type.CATEGORICAL', 
                'Type.CATEGORICAL', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(3, ground_truth)

{'X': array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 1., 0., 1.],
       [0., 1., 0., ..., 1., 0., 1.],
       [0., 1., 0., ..., 1., 0., 1.]], dtype=float32), 'T': array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]), 'R': array([102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102,
       102, 103, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102,
       102, 102, 102, 102, 102, 102, 102, 102, 102, 102, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (3196, 37)
	meta-types:[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
	maximal discrete cardinality: [102 102 102 102 102 102 102 102 102 102 102 102 102 102 103 102 102 102
 102 102 102 102 102 102 102 102 102 102 102 102 102 102 102 102 102 102
 102]
	meta types [<MetaType.DISCRETE: 3> <MetaType.DISCRETE: 3> <M

        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101], dtype=int64)]
	translated discrete features (starting from 0)
MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gaussian'>, {'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EB91219E8>})])), (<Type.POSITIVE: (3, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gamma'>, {'params': {'alpha': None, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EB9121A20>}), (<class 'bayesian.spn.structure.leaves.parametric.Parametric.Exponential'>, {'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EBC87FF98>})]))])
MT MetaType.DISCRETE OrderedDict([(<Type.CATEGORICAL: (4, <MetaType.DISCRETE: 3>)>, {<class 'bayesian.spn.structure.leaves.parametric.P

       8.22368414e-11, 8.22368414e-11])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3BA7860>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3BA7940>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3BA7A90>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([5.05050502e-11, 9.99999995e-01, 5.05050502e-11, 5.05050502e-11,
       5.05050502e-11, 5.05050502e-11, 5.05050502e-11, 5.05050502e-11,
       5.05050502e-11, 5.05050502e-11, 5.05050502e-11, 5.05050502e-11,
       5.05050502e-11, 5.05050502e-11, 5.05050502e-11, 5.05050502e-11,
       5.050

       8.22368414e-11, 8.22368414e-11])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B84DA0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B84BE0>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B84C50>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([4.44078944e-02, 9.55592097e-01, 8.22368414e-11, 8.22368414e-11,
       8.22368414e-11, 8.22368414e-11, 8.22368414e-11, 8.22368414e-11,
       8.22368414e-11, 8.22368414e-11, 8.22368414e-11, 8.22368414e-11,
       8.22368414e-11, 8.22368414e-11, 8.22368414e-11, 8.22368414e-11,
       8.223

       8.22368414e-11, 8.22368414e-11])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B8F7F0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B8F0B8>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B8F630>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([4.44901312e-01, 5.55098680e-01, 8.22368414e-11, 8.22368414e-11,
       8.22368414e-11, 8.22368414e-11, 8.22368414e-11, 8.22368414e-11,
       8.22368414e-11, 8.22368414e-11, 8.22368414e-11, 8.22368414e-11,
       8.22368414e-11, 8.22368414e-11, 8.22368414e-11, 8.22368414e-11,
       8.223

...

       5.05050502e-11, 5.05050502e-11])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B70C50>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B70FD0>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B701D0>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([1.86868686e-02, 9.81313126e-01, 5.05050502e-11, 5.05050502e-11,
       5.05050502e-11, 5.05050502e-11, 5.05050502e-11, 5.05050502e-11,
       5.05050502e-11, 5.05050502e-11, 5.05050502e-11, 5.05050502e-11,
       5.05050502e-11, 5.05050502e-11, 5.05050502e-11, 5.05050502e-11,
       5.050

       5.05050502e-11, 5.05050502e-11])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B77518>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B77550>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B77780>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([3.82323230e-01, 6.17676765e-01, 5.05050502e-11, 5.05050502e-11,
       5.05050502e-11, 5.05050502e-11, 5.05050502e-11, 5.05050502e-11,
       5.05050502e-11, 5.05050502e-11, 5.05050502e-11, 5.05050502e-11,
       5.05050502e-11, 5.05050502e-11, 5.05050502e-11, 5.05050502e-11,
       5.050

       5.05050502e-11, 5.05050502e-11])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B77E10>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B77748>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B77F98>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([5.55555553e-01, 4.44444442e-01, 5.05050502e-11, 5.05050502e-11,
       5.05050502e-11, 5.05050502e-11, 5.05050502e-11, 5.05050502e-11,
       5.05050502e-11, 5.05050502e-11, 5.05050502e-11, 5.05050502e-11,
       5.05050502e-11, 5.05050502e-11, 5.05050502e-11, 5.05050502e-11,
       5.050

       1.92307689e-10, 1.92307689e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B70F98>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B70DD8>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B709E8>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([9.99999981e-01, 1.92307689e-10, 1.92307689e-10, 1.92307689e-10,
       1.92307689e-10, 1.92307689e-10, 1.92307689e-10, 1.92307689e-10,
       1.92307689e-10, 1.92307689e-10, 1.92307689e-10, 1.92307689e-10,
       1.92307689e-10, 1.92307689e-10, 1.92307689e-10, 1.92307689e-10,
       1.923

 5.05050502e-11 5.05050502e-11])) + 0.3333333333333333*(Geometric(V36|p=0.5)) + 0.3333333333333333*(Poisson(V36|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
Done iteration 1/1 in 0.3674310459850858


------------------
Done in 0.37368279771499147
Dumped id to ./exp/temp/data.mat\20181029-152536\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-152536\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-152536
['Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Ty

In [73]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.10810810810810811


In [41]:
# 1510
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE',
                'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1510, ground_truth)

{'X': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 4.601e-01, 1.189e-01,
        1.000e+00],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 2.750e-01, 8.902e-02,
        1.000e+00],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 3.613e-01, 8.758e-02,
        1.000e+00],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 2.218e-01, 7.820e-02,
        1.000e+00],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 4.087e-01, 1.240e-01,
        1.000e+00],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 2.871e-01, 7.039e-02,
        0.000e+00]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 1, 1, 4]), 'R': array([  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (569, 31)
	meta-types:[1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 2 2 1 1 4]
	maxi

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC2285BA8>}
gamma fit params (52.57226377910289, 0, 0.003284387269548723)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC0859630>}
{'params': {'alpha': 52.57226377910289, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0859710>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0859E80>}
gamma fit params (104.56015159932574, 0, 0.0005934724068918708)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B2E908>}
{'params': {'alpha': 104.56015159932574, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC40F3588>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC40F3470>}

{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B270B8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B27128>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B27198>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B27390>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B27400>}
gamma fit params (40.17960369348685, 0, 0.006702608586109791)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B27588>}
{'params': {'alpha': 40.17960369348685, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B275C0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Poster

{'params': {'alpha': 69.959569224947, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B07780>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B07978>}
gamma fit params (3.9164005526527883, 0, 0.14765090446519819)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B07A90>}
{'params': {'alpha': 3.9164005526527883, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B07710>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B07C50>}
gamma fit params (6.70682253856718, 0, 0.18083661311191762)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B07DD8>}
{'params': {'alpha': 6.70682253856718, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC4912D68>}
gamma fit params (21.48990769214565, 0, 0.014842222109061783)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC4912F60>}
{'params': {'alpha': 21.48990769214565, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC4912FD0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC4907080>}
gamma fit params (22.53152301093338, 0, 0.0041247123096782405)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC4907208>}
{'params': {'alpha': 22.53152301093338, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC4907198>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC4907320>}



 4.20168049e-10 4.20168049e-10])) + 0.3333333333333333*(Geometric(V30|p=0.5)) + 0.3333333333333333*(Poisson(V30|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Done iteration 1/1 in 0.20627583184966625


------------------
Done in 0.21417647964437947
Dumped id to ./exp/temp/data.mat\20181029-144011\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-144011\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-144011
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIV

In [42]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [43]:
# 1489
ground_truth = ['Type.REAL', 'Type.REAL', 'Type.REAL', 'Type.REAL', 'Type.REAL',
                'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1489, ground_truth)

{'X': array([[ 0.489927, -0.451528, -1.04799 , -0.598693, -0.020418,  0.      ],
       [-0.641265,  0.109245,  0.29213 , -0.916804,  0.240223,  0.      ],
       [ 0.870593, -0.459862,  0.578159,  0.806634,  0.835248,  0.      ],
       ...,
       [ 0.246882, -0.793228,  1.190101,  1.423194, -1.303036,  1.      ],
       [-0.778907, -0.383111,  1.727029, -1.432389, -1.208085,  0.      ],
       [-0.794604, -0.640053,  0.632221,  0.72028 , -1.231182,  1.      ]],
      dtype=float32), 'T': array([2, 2, 2, 2, 2, 4]), 'R': array([  1,   1,   1,   1,   1, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (5404, 6)
	meta-types:[2 2 2 2 2 4]
	maximal discrete cardinality: [  1   1   1   1   1 102]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([-2.933743 ,  3.8285749]), array([-3.03870893,  3.66476297]), array([-2.79586911,  2.63086796]), array([-2.48534012,  3.04694796]), array([-2.

       3.79506640e-11, 3.79506640e-11])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B2E908>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B04F98>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC052E748>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B26F60>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B26D68>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B26B70>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B26CF8>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B26FD0>}
{'params': {'l': 

In [44]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.5


In [77]:
# 37
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(37, ground_truth)

{'X': array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]],
      dtype=float32), 'T': array([2, 2, 2, 2, 2, 2, 1, 1, 4]), 'R': array([  1,   1,   1,   1,   1,   1,   1,   1, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (768, 9)
	meta-types:[2 2 2 2 2 2 1 1 4]
	maximal discrete cardinality: [  1   1   1   1   1   1   1   1 102]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([ 0., 17.]), array([  0., 199.]), array([  0., 122.]), array([ 0., 99.]), array([  0.

       2.69541772e-10, 2.69541772e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B7ADD8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B7AD68>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B7A6A0>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B73BA8>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B73940>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B73908>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B73FD0>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B73E80>}
{'params': {'l': 

 2.51889162e-10 2.51889162e-10])) + 0.3333333333333333*(Geometric(V8|p=0.5)) + 0.3333333333333333*(Poisson(V8|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8]
Done iteration 1/1 in 0.05622392142777244


------------------
Done in 0.059795674262659304
Dumped id to ./exp/temp/data.mat\20181029-153021\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-153021\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-153021
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']


In [78]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [183]:
# 1479
ground_truth = []
for i in range(100):
    ground_truth.append('Type.POSITIVE')
ground_truth.append('Type.CATEGORICAL')
accuracy = evaluate_bayesian(1479, ground_truth)

{'X': array([[3.902000e+01, 3.649000e+01, 3.820000e+01, ..., 3.750000e+01,
        3.910000e+01, 0.000000e+00],
       [1.830000e+00, 1.710000e+00, 1.770000e+00, ..., 1.750000e+00,
        1.690000e+00, 1.000000e+00],
       [6.817769e+04, 6.613842e+04, 7.298188e+04, ..., 6.740427e+04,
        7.492024e+04, 1.000000e+00],
       ...,
       [1.016065e+04, 9.048630e+03, 8.994940e+03, ..., 8.997600e+03,
        9.305770e+03, 1.000000e+00],
       [3.481000e+01, 3.507000e+01, 3.498000e+01, ..., 3.283000e+01,
        3.482000e+01, 1.000000e+00],
       [8.489430e+03, 7.672980e+03, 9.132140e+03, ..., 8.389310e+03,
        8.712800e+03, 0.000000e+00]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1

       51], dtype=int64)]
	translated discrete features (starting from 0)
MT MetaType.REAL OrderedDict([(<Type.REAL: (1, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gaussian'>, {'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE2158198>})])), (<Type.POSITIVE: (3, <MetaType.REAL: 1>)>, OrderedDict([(<class 'bayesian.spn.structure.leaves.parametric.Parametric.Gamma'>, {'params': {'alpha': None, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE2158710>}), (<class 'bayesian.spn.structure.leaves.parametric.Parametric.Exponential'>, {'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE5D04128>})]))])
MT MetaType.DISCRETE OrderedDict([(<Type.CATEGORICAL: (4, <MetaType.DISCRETE: 3>)>, {<class 'bayesian.spn.structure.leaves.parametric.Parametric.Categorical'>: {'params': {'p': None}, 'p

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

...

{'params': {'alpha': 1.7941980930653831, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9C189E8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9C18898>}
gamma fit params (1.8013802992429535, 0, 57.967807068874)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9C18F98>}
{'params': {'alpha': 1.8013802992429535, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9C184E0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9C18710>}
gamma fit params (1.825218263514455, 0, 56.857390434555995)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9C18668>}
{'params': {'alpha': 1.825218263514455, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE98EFE48>}
{'params': {'alpha': 1.8053480583366686, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE98EFDD8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE98EF240>}
gamma fit params (1.7960801573791538, 0, 58.30348799748664)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9700278>}
{'params': {'alpha': 1.7960801573791538, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9700048>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9700160>}
gamma fit params (1.794686572034761, 0, 58.43656083580609)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object a

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E9240>}
gamma fit params (1.7217319919687364, 0, 3.256745266208983)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBEB1E9B38>}
{'params': {'alpha': 1.7217319919687364, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E97B8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E9630>}
gamma fit params (1.7236174215686124, 0, 3.2694508191991907)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBEB1E9A90>}
{'params': {'alpha': 1.7236174215686124, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E98D0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E9F60>}
gam

{'params': {'alpha': 1.7657624529922589, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE98FAC18>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE98FA550>}
gamma fit params (1.7465505733488662, 0, 3.219387981609981)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE98FA048>}
{'params': {'alpha': 1.7465505733488662, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE98FAE10>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE98FAC88>}
gamma fit params (1.7418494713027437, 0, 3.224357564176131)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE98FA7B8>}
{'params': {'alpha': 1.7418494713027437, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma obj

 1.65016500e-10 1.65016500e-10 1.65016500e-10 1.65016500e-10])) + 0.3333333333333333*(Geometric(V100|p=0.5)) + 0.3333333333333333*(Poisson(V100|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
Done iteration 1/1 in 1.6532146959789316


------------------
Done in 1.7364537484063476
Dumped id to ./exp/temp/data.mat\20181028-191733\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181028-191733\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181028-191733
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.PO

In [184]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9900990099009901


In [115]:
# 1487
ground_truth = []
for i in range(73):
    ground_truth.append('Type.POSITIVE')
for i in [54, 55, 60, 62, 65, 70]:
    ground_truth[i] = 'Type.REAL'
ground_truth[72] = ('Type.CATEGORICAL')
accuracy = evaluate_bayesian(1479, ground_truth)

{'X': array([[3.902000e+01, 3.649000e+01, 3.820000e+01, ..., 3.750000e+01,
        3.910000e+01, 0.000000e+00],
       [1.830000e+00, 1.710000e+00, 1.770000e+00, ..., 1.750000e+00,
        1.690000e+00, 1.000000e+00],
       [6.817769e+04, 6.613842e+04, 7.298188e+04, ..., 6.740427e+04,
        7.492024e+04, 1.000000e+00],
       ...,
       [1.016065e+04, 9.048630e+03, 8.994940e+03, ..., 8.997600e+03,
        9.305770e+03, 1.000000e+00],
       [3.481000e+01, 3.507000e+01, 3.498000e+01, ..., 3.283000e+01,
        3.482000e+01, 1.000000e+00],
       [8.489430e+03, 7.672980e+03, 9.132140e+03, ..., 8.389310e+03,
        8.712800e+03, 0.000000e+00]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon

...

{'params': {'alpha': 1.7673349768863695, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9CC3630>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9CC3D30>}
gamma fit params (1.7797000835098105, 0, 3.166049151735551)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBE9FABA20>}
{'params': {'alpha': 1.7797000835098105, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9FAB0B8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBE9FAB080>}
gamma fit params (1.755716770340547, 0, 3.2028830513315385)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBED629EB8>}
{'params': {'alpha': 1.755716770340547, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma obje

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBEB11D780>}
{'params': {'alpha': 1.783132565567605, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB11D438>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB11D748>}
gamma fit params (1.793993452897549, 0, 58.309824066590494)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBEB11DAC8>}
{'params': {'alpha': 1.793993452897549, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB11D6A0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB11D8D0>}
gamma fit params (1.7944288152651895, 0, 58.569859147554475)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object a

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E9358>}
gamma fit params (1.7776011109010725, 0, 58.92994406822095)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBEB1E9978>}
{'params': {'alpha': 1.7776011109010725, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E93C8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E9B00>}
gamma fit params (1.7646087863499302, 0, 59.71978961279681)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x000000CBEB1E9748>}
{'params': {'alpha': 1.7646087863499302, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E9908>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x000000CBEB1E98D0>}
gamm

(0.5*(((0.5066006600660066*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=1.6545988685199116;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=1.6570433180904642;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=1.6414960718966134;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V3|alpha=1.656301074955083;beta=1)) + 0.3333333333333333*(Exponential(V3|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V4|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V4|alpha=1.6658529706781784;beta=1)) + 0.3333333333333333*(Exponential(V4|l=2))) * TypeMixtureUncons

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
Done iteration 1/1 in 1.436130443192269


------------------
Done in 1.4589497120250599
Dumped id to ./exp/temp/data.mat\20181028-181948\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181028-181948\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181028-181948
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIV

In [116]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9041095890410958


In [80]:
# 1063
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1063, ground_truth)

{'X': array([[  1.1,   1.4,   1.4, ...,   1.2,   1.4,   0. ],
       [  1. ,   1. ,   1. , ...,   1. ,   1. ,   1. ],
       [415. ,  59. ,  50. , ..., 467. , 106. ,   1. ],
       ...,
       [  4. ,   1. ,   1. , ...,   1. ,   1. ,   1. ],
       [  4. ,   1. ,   1. , ...,   2. ,   1. ,   1. ],
       [  3. ,   1. ,   1. , ...,   0. ,   1. ,   1. ]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 1, 2, 1, 2, 1, 4]), 'R': array([  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1, 108,   1,   1,   1,   1,   1, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (522, 22)
	meta-types:[1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 4 1 2 1 2 1 4]
	maximal discrete cardinality: [  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1 108   1   1
   1   1   1 102]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.RE

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC490C470>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B70898>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B70198>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B70518>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B707B8>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B70E48>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B70588>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B70EF0>}
{'params': {'alpha

       3.47222210e-10, 3.47222210e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B7A320>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B7AA20>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC491D908>}
gamma fit params (1.6008622923605138, 0, 46.414622393195636)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC492DB70>}
{'params': {'alpha': 1.6008622923605138, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC492DCF8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC492D4A8>}
gamma fit params (1.3490934604162566, 0, 6.927728967917563)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC492D198>}
{'para

       4.27350408e-10, 4.27350408e-10, 4.27350408e-10, 4.27350408e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B840B8>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B84160>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B84FD0>}
gamma fit params (10.706506012229827, 0, 1.3918368312395732)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC491D9B0>}
{'params': {'alpha': 10.706506012229827, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC491D940>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC491DC50>}
gamma fit params (2.516805979358195, 0, 11.250863017839848)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma objec

 4.27350409e-10 4.27350409e-10])) + 0.3333333333333333*(Geometric(V21|p=0.5)) + 0.3333333333333333*(Poisson(V21|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
Done iteration 1/1 in 0.1327876488721813


------------------
Done in 0.18472609850732624
Dumped id to ./exp/temp/data.mat\20181029-153333\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-153333\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-153333
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']


In [81]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9545454545454546


In [83]:
# 1471
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1471, ground_truth)

{'X': array([[4.32923e+03, 4.00923e+03, 4.28923e+03, ..., 4.63590e+03,
        4.39385e+03, 0.00000e+00],
       [4.32462e+03, 4.00462e+03, 4.29385e+03, ..., 4.63282e+03,
        4.38410e+03, 0.00000e+00],
       [4.32769e+03, 4.00667e+03, 4.29538e+03, ..., 4.62872e+03,
        4.38923e+03, 0.00000e+00],
       ...,
       [4.27744e+03, 3.99077e+03, 4.24667e+03, ..., 4.59179e+03,
        4.33949e+03, 1.00000e+00],
       [4.28462e+03, 3.99179e+03, 4.25128e+03, ..., 4.59641e+03,
        4.35077e+03, 1.00000e+00],
       [4.28769e+03, 3.99744e+03, 4.26000e+03, ..., 4.59795e+03,
        4.35077e+03, 1.00000e+00]], dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4]), 'R': array([  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (14980, 15)
	meta-types:[1 1 1 1 1 1 1 1 1 1 1 1 1 1 4]
	maximal discrete cardinality: [  1   1   1   1   1   1   1   1   1   1   1   1   1   1 102]
	meta types 

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B847B8>}
gamma fit params (17737.705654687892, 0, 0.24439487366379956)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B84F98>}
{'params': {'alpha': 17737.705654687892, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B84C50>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B843C8>}
gamma fit params (30844.736766757287, 0, 0.1495823413943941)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B84EF0>}
{'params': {'alpha': 30844.736766757287, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B84E10>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B848D0>}
g

{'params': {'alpha': 14825.2778404572, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC492D7B8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC492D828>}
gamma fit params (73435.81024645608, 0, 0.06288973860405891)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC492D860>}
{'params': {'alpha': 73435.81024645608, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC492DB38>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC492D2E8>}
gamma fit params (4510.072066995253, 0, 0.9321950896394744)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B707B8>}
{'params': {'alpha': 4510.072066995253, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B77BE0>}
gamma fit params (224068.14791104966, 0, 0.019433627568626828)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B77A90>}
{'params': {'alpha': 224068.14791104966, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B776A0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B770B8>}
gamma fit params (2.327737257460709, 0, 2417.8791357819505)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B77908>}
{'params': {'alpha': 2.327737257460709, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B77710>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B77C18>}
ga

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B75E80>}
{'params': {'alpha': 37227.41606406301, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B757F0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B75C88>}
gamma fit params (41045.16920429063, 0, 0.10008737633093612)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B75AC8>}
{'params': {'alpha': 41045.16920429063, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B75C50>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B75320>}
gamma fit params (56798.994840074636, 0, 0.08188245505560988)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object

{'params': {'alpha': 182924.84624843457, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC491CC18>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC491CCC0>}
gamma fit params (157579.1250867277, 0, 0.02738984056054774)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC491C390>}
{'params': {'alpha': 157579.1250867277, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC491C5C0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC491C1D0>}
gamma fit params (101228.40182327972, 0, 0.04328668936836785)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC491CBA8>}
{'params': {'alpha': 101228.40182327972, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma o

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B7CC88>}
gamma fit params (52.256644037560015, 0, 86.03858615089234)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B7C9E8>}
{'params': {'alpha': 52.256644037560015, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B7C9B0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B7CE10>}
gamma fit params (335.8159932102581, 0, 12.677962657397597)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B7CDD8>}
{'params': {'alpha': 335.8159932102581, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B7C748>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B7CF98>}
gamma

 1.23274162e-11 1.23274162e-11])) + 0.3333333333333333*(Geometric(V14|p=0.5)) + 0.3333333333333333*(Poisson(V14|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Done iteration 1/1 in 1.550991429845908


------------------
Done in 1.5711792547795085
Dumped id to ./exp/temp/data.mat\20181029-153533\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-153533\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-153533
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']


In [84]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [86]:
# 1467
ground_truth = ['Type.CATEGORICAL', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1467, ground_truth)

{'X': array([[1.00000e+00, 1.00000e+00, 5.02000e+02, ..., 7.96997e-01,
        8.69893e-01, 0.00000e+00],
       [1.00000e+00, 2.00000e+00, 2.49000e+02, ..., 4.38447e-01,
        5.12256e-01, 1.00000e+00],
       [1.00000e+00, 3.00000e+00, 2.02000e+02, ..., 2.85636e-01,
        3.65858e-01, 1.00000e+00],
       ...,
       [3.00000e+00, 1.78000e+02, 5.09000e+02, ..., 4.61632e-01,
        6.52817e-01, 1.00000e+00],
       [3.00000e+00, 1.79000e+02, 4.21000e+02, ..., 2.01469e-01,
        5.36535e-01, 1.00000e+00],
       [3.00000e+00, 1.80000e+02, 1.80000e+01, ..., 7.60344e-01,
        7.62439e-01, 1.00000e+00]], dtype=float32), 'T': array([4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4]), 'R': array([103,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (540, 21)
	meta-types:[4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4]
	maximal discrete cardinality: [103   1   1   1

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B27F98>}
gamma fit params (1.7158410464450689, 0, 0.27438644947794966)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B8EC88>}
{'params': {'alpha': 1.7158410464450689, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B8E8D0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B8E978>}
gamma fit params (1.8185042852032607, 0, 0.27088165266250097)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B8EDA0>}
{'params': {'alpha': 1.8185042852032607, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B8E1D0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B7A320>}


       3.73134314e-10, 3.73134314e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC4935B38>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC4935D30>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC4935390>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([3.27205870e-01, 3.38235282e-01, 3.34558811e-01, 3.67647045e-10,
       3.67647045e-10, 3.67647045e-10, 3.67647045e-10, 3.67647045e-10,
       3.67647045e-10, 3.67647045e-10, 3.67647045e-10, 3.67647045e-10,
       3.67647045e-10, 3.67647045e-10, 3.67647045e-10, 3.67647045e-10,
       3.

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B8F668>}
gamma fit params (1.749449414961663, 0, 0.28119459892737775)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B8FC50>}
{'params': {'alpha': 1.749449414961663, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B8F3C8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B8FEB8>}
gamma fit params (1.7369788502103194, 0, 0.2741599742657307)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B8F400>}
{'params': {'alpha': 1.7369788502103194, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B8F240>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B8F5C0>}
gam

 3.67647045e-10 3.67647045e-10])) + 0.3333333333333333*(Geometric(V20|p=0.5)) + 0.3333333333333333*(Poisson(V20|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Done iteration 1/1 in 0.12837664465314447


------------------
Done in 0.17865356602942484
Dumped id to ./exp/temp/data.mat\20181029-153928\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-153928\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-153928
['Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.COUNT']


In [87]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9047619047619048


In [92]:
# 1480
ground_truth = ['Type.POSITIVE', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1480, ground_truth)

{'X': array([[65.  ,  0.  ,  0.7 , ...,  3.3 ,  0.9 ,  0.  ],
       [62.  ,  1.  , 10.9 , ...,  3.2 ,  0.74,  0.  ],
       [62.  ,  1.  ,  7.3 , ...,  3.3 ,  0.89,  0.  ],
       ...,
       [52.  ,  1.  ,  0.8 , ...,  3.2 ,  1.  ,  0.  ],
       [31.  ,  1.  ,  1.3 , ...,  3.4 ,  1.  ,  0.  ],
       [38.  ,  1.  ,  1.  , ...,  4.4 ,  1.5 ,  1.  ]], dtype=float32), 'T': array([1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4]), 'R': array([  1, 102,   1,   1,   1,   1,   1,   1,   1,   1, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (583, 11)
	meta-types:[1 4 1 1 1 1 1 1 1 1 4]
	maximal discrete cardinality: [  1 102   1   1   1   1   1   1   1   1 102]
	meta types [<MetaType.REAL: 1> <MetaType.DISCRETE: 3> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([ 4., 90.]), array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
      

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC4928940>}
{'params': {'alpha': 4.089194572360055, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC4928FD0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC4937940>}
gamma fit params (2.7768325217549656, 0, 13.027135734967553)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC49201D0>}
{'params': {'alpha': 2.7768325217549656, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1C2B470>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1C2B710>}
gamma fit params (33.81536829619694, 0, 0.19607446777688886)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC2067C50>}
{'params': {'alpha': 0.8414985370842705, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC2067518>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC2067EB8>}
gamma fit params (2.720273786426483, 0, 150.6583646267386)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC2067BA8>}
{'params': {'alpha': 2.720273786426483, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC2067E80>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC2067828>}
gamma fit params (0.9420284467482022, 0, 157.12901294111848)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object a

 3.99999984e-10 3.99999984e-10])) + 0.3333333333333333*(Geometric(V10|p=0.5)) + 0.3333333333333333*(Poisson(V10|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Done iteration 1/1 in 0.0731005325260412


------------------
Done in 0.07712112300214358
Dumped id to ./exp/temp/data.mat\20181029-154338\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-154338\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-154338
['Type.POSITIVE', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']


In [93]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [95]:
# 1068
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1068, ground_truth)

{'X': array([[  1.1,   1.4,   1.4, ...,   1.2,   1.4,   0. ],
       [  1. ,   1. ,   1. , ...,   1. ,   1. ,   1. ],
       [ 91. ,   9. ,   3. , ..., 126. ,  17. ,   1. ],
       ...,
       [  5. ,   3. ,   3. , ...,   6. ,   5. ,   0. ],
       [ 18. ,   8. ,   5. , ...,  50. ,  15. ,   0. ],
       [ 26. ,  18. ,  13. , ..., 109. ,  35. ,   0. ]], dtype=float32), 'T': array([2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 4]), 'R': array([  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (1109, 22)
	meta-types:[2 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 1 2 1 2 1 4]
	maximal discrete cardinality: [  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1 102]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.R

       1.85185182e-10, 1.85185182e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC3B1E748>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC3B1E2E8>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1C2B668>}
gamma fit params (4.771799535316566, 0, 0.26532565632167593)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B0A828>}
{'params': {'alpha': 4.771799535316566, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B0AA20>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B0A978>}
gamma fit params (5.9188847553123916, 0, 0.23112699710603687)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC2067A20>}
{'par

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B07A58>}
gamma fit params (1.9768689036940403, 0, 0.38885212470067754)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B07048>}
{'params': {'alpha': 1.9768689036940403, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B07588>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B07C18>}
gamma fit params (0.7422352972710606, 0, 8776.048619647914)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC40F3940>}
{'params': {'alpha': 0.7422352972710606, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC40F3390>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC40F3A90>}
ga

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC059F748>}
{'params': {'alpha': 7.0241312689044495, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC059FA58>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC059F6D8>}
gamma fit params (11.867677058839584, 0, 4.475146128242296)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC059F588>}
{'params': {'alpha': 11.867677058839584, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0526E48>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0526CF8>}
gamma fit params (9.388474207609136, 0, 4.302856820039148)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object a

{'params': {'alpha': 7.951623517034152, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0A27518>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0A27B70>}
gamma fit params (5.5295353454432865, 0, 0.02333453318866247)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC2078DA0>}
{'params': {'alpha': 5.5295353454432865, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC2078FD0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC2078E80>}
gamma fit params (5.440652142900486, 0, 1.7191745010542587)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC20784A8>}
{'params': {'alpha': 5.440652142900486, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma obj

 1.75746921e-10 1.75746921e-10])) + 0.3333333333333333*(Geometric(V21|p=0.5)) + 0.3333333333333333*(Poisson(V21|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
Done iteration 1/1 in 0.258120724298351


------------------
Done in 0.29435972540795774
Dumped id to ./exp/temp/data.mat\20181029-154550\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-154550\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-154550
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']


In [96]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 1.0


In [98]:
# 1492
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']

accuracy = evaluate_bayesian(1492, ground_truth)

{'X': array([[5.79e-04, 6.09e-04, 5.51e-04, ..., 6.17e-04, 6.14e-04, 1.20e+01],
       [6.30e-04, 6.61e-04, 7.19e-04, ..., 6.71e-04, 6.67e-04, 1.20e+01],
       [6.16e-04, 6.15e-04, 6.06e-04, ..., 6.07e-04, 6.02e-04, 1.20e+01],
       ...,
       [7.01e-04, 6.42e-04, 6.46e-04, ..., 6.22e-04, 6.63e-04, 2.00e+00],
       [7.09e-04, 7.76e-04, 7.16e-04, ..., 6.83e-04, 7.02e-04, 2.00e+00],
       [8.50e-04, 7.70e-04, 7.64e-04, ..., 8.10e-04, 7.97e-04, 2.00e+00]],
      dtype=float32), 'T': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]), 'R': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

Loaded bayesian/data/temp/data.mat
	with shape: (1600, 65)
	meta-typ

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC02F5710>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC02F55C0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC02F5400>}


D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)


{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC20784E0>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC2078048>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0526780>}
gamma fit params (12.849018071847684, 0, 3.3961159326277766e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B1EB00>}
{'params': {'alpha': 12.849018071847684, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B1EBA8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B1E208>}
gamma fit params (12.833858922022932, 0, 3.213096235463534e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0

...

gamma fit params (43.58490522605466, 0, 1.71685740514581e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC0318630>}
{'params': {'alpha': 43.58490522605466, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC03186D8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC03189E8>}
gamma fit params (40.95187465849605, 0, 1.9428391857701196e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC0318518>}
{'params': {'alpha': 40.95187465849605, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC03184E0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0318048>}
gamma fit params (40.03240551367586, 0, 2.1108929230193512e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior':

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC07478D0>}
{'params': {'alpha': 34.393381802419846, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0747198>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0747048>}
gamma fit params (26.060813696877013, 0, 1.7970398641222116e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC0747550>}
{'params': {'alpha': 26.060813696877013, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0747DD8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0747B70>}
gamma fit params (21.93591314803734, 0, 2.0669261829378848e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamm

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC066D6A0>}
{'params': {'alpha': 29.747838486363104, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC066D198>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC066DEB8>}
gamma fit params (28.78419871880063, 0, 3.3725270255702386e-05)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC066D3C8>}
{'params': {'alpha': 28.78419871880063, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC066D668>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC066D898>}
True True True
True True True


(0.436875*(((0.43204577968526464*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V0|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V0|alpha=12.849018071847684;beta=1)) + 0.3333333333333333*(Exponential(V0|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=12.833858922022932;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=12.540056076017919;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V3|alpha=11.867288992163187;beta=1)) + 0.3333333333333333*(Exponential(V3|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V4|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V4|alpha=11.441125187891954;beta=1)) + 0.3333333333333333*(Exponential(V4|l=2))) * TypeMixtur

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64]
Done iteration 1/1 in 1.014366357309882


------------------
Done in 1.0298877708519285
Dumped id to ./exp/temp/data.mat\20181029-154821\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-154821\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-154821
['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type

In [99]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.9846153846153847


In [101]:
# 1050
ground_truth = ['Type.POSITIVE', 'Type.COUNT', 'Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 
                'Type.POSITIVE', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.POSITIVE', 
                'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT', 'Type.COUNT',
                'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL']

accuracy = evaluate_bayesian(1050, ground_truth)

{'X': array([[ 2.  ,  1.  ,  0.  , ...,  0.  , 10.  ,  0.  ],
       [ 1.  ,  1.  ,  4.  , ...,  0.  , 14.  ,  0.  ],
       [27.  , 19.  ,  1.  , ..., 30.91, 42.  ,  0.  ],
       ...,
       [ 0.  ,  1.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  7.  ,  0.  , ...,  0.  , 11.  ,  0.  ],
       [ 1.  ,  9.  ,  0.  , ...,  0.  , 11.  ,  0.  ]], dtype=float32), 'T': array([2, 1, 4, 4, 2, 2, 1, 1, 2, 2, 1, 1, 1, 4, 2, 2, 4, 2, 2, 2, 2, 1,
       2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 4]), 'R': array([  1,   1, 120, 125,   1,   1,   1,   1,   1,   1,   1,   1,   1,
       125,   1,   1, 108,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (1563, 38)
	meta-types:[2 1 4 4 2 2 1 1 2 2 1 1 1 4 2 2 4 2 2 2 2 1 2 2 2 1 2 2 1 1 2 1 2 1 1 2 2
 4]
	maximal discrete cardinality: [  1   1 120 125   1   1   1   1   1   1   1   1   1 125   1   1 108   1
   1   1   1   1   1 

D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anaconda3\lib\site-packages\sklearn\cross_decomposition\pls_.py:79: ConvergenceWarning: Maximum number of iterations reached
  ConvergenceWarning)
D:\Programfiles\Anacon





CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
{'params': {'p': array([1.25794153e-01, 1.77890722e-01, 1.55019057e-01, 1.51207113e-01,
       1.29606097e-01, 6.60736967e-02, 4.57433285e-02, 3.55781444e-02,
       3.30368484e-02, 2.03303683e-02, 8.89453620e-03, 1.14358322e-02,
       1.14358322e-02, 1.01651842e-02, 5.08259217e-03, 1.27064801e-10,
       3.81194416e-03, 1.27064801e-10, 1.27064801e-10, 3.81194416e-03,
       1.27064814e-03, 1.27064801e-10, 1.27064814e-03, 1.27064801e-10,
       2.54129615e-03, 1.27064801e-10, 1.27064801e-10, 1.27064801e-10,
       1.27064801e-10, 1.27064801e-10, 1.27064801e-10, 1.27064801e-10,
       1.27064801e-10, 1.27

       1.27064801e-10, 1.27064801e-10, 1.27064801e-10, 1.27064801e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC1F4C5C0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC052EC50>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC052E860>}
gamma fit params (9.759365636087132, 0, 1.7930842360570531)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC052EBA8>}
{'params': {'alpha': 9.759365636087132, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC052E4A8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC052EDD8>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 

       1.28865977e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC40D08D0>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC1F3E748>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1F3EC18>}
gamma fit params (6.201562973772353, 0, 0.21693899270749445)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B175C0>}
{'params': {'alpha': 6.201562973772353, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B179E8>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC3B173C8>}
gamma fit params (8.939351907243006, 0, 0.09599914104853517)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC3B17518>}
{'params': {'alpha': 8

...

{'params': {'alpha': 2.0830658866285034, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0632BE0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC06320F0>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC4920E10>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC4920F98>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1DFAD30>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC1DFAB00>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1DFA518>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1DFA5F8>}
{'params': {'m

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC1F36470>}
{'params': {'alpha': 3.8124361740407435, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1F365C0>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1F36F60>}
gamma fit params (5.698826587531501, 0, 0.02280343653715467)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC047D080>}
{'params': {'alpha': 5.698826587531501, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC047D208>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC047DA20>}
gamma fit params (1.3333156914548836, 0, 102.17249007192552)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object

{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC059F748>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC059F588>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC059F6D8>}
gamma fit params (10.634946448132684, 0, 0.3713159018187778)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC04C22B0>}
{'params': {'alpha': 10.634946448132684, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC04C2E48>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC04C2518>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC04C26D8>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <ba

{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0927390>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC0927BA8>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0927E10>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0927F28>}
gamma fit params (661.1241146148706, 0, 0.0015163192891681642)
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC09270B8>}
{'params': {'alpha': 661.1241146148706, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0927908>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC0927860>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteri

 1.28865978e-10 1.28865978e-10])) + 0.3333333333333333*(Geometric(V37|p=0.5)) + 0.3333333333333333*(Poisson(V37|mean=2))))))
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]
Done iteration 1/1 in 0.591538403843515


------------------
Done in 0.631779441893741
Dumped id to ./exp/temp/data.mat\20181029-155951\global-W.pklz
Dumped id to ./exp/temp/data.mat\20181029-155951\global-type-W.pklz
seeeeeeeeeeeeeeeeeeeeeeeeeee
./exp/temp/data.mat\20181029-155951
['Type.POSITIVE', 'Type.POSITIVE', 'Type.COUNT', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.CATEGORICAL', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.COUNT', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.

In [102]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.6842105263157895


In [104]:
# 1462
ground_truth = ['Type.POSITIVE', 'Type.POSITIVE', 'Type.POSITIVE', 'Type.REAL', 'Type.CATEGORICAL']
accuracy = evaluate_bayesian(1462, ground_truth)

{'X': array([[  3.6216 ,   8.6661 ,  -2.8073 ,  -0.44699,   0.     ],
       [  4.5459 ,   8.1674 ,  -2.4586 ,  -1.4621 ,   0.     ],
       [  3.866  ,  -2.6383 ,   1.9242 ,   0.10645,   0.     ],
       ...,
       [ -3.7503 , -13.4586 ,  17.5932 ,  -2.7771 ,   1.     ],
       [ -3.5637 ,  -8.3827 ,  12.393  ,  -1.2823 ,   1.     ],
       [ -2.5419 ,  -0.65804,   2.6842 ,   1.1952 ,   1.     ]],
      dtype=float32), 'T': array([2, 2, 2, 2, 4]), 'R': array([  1,   1,   1,   1, 102])}

Loaded bayesian/data/temp/data.mat
	with shape: (1372, 5)
	meta-types:[2 2 2 2 4]
	maximal discrete cardinality: [  1   1   1   1 102]
	meta types [<MetaType.REAL: 1> <MetaType.REAL: 1> <MetaType.REAL: 1>
 <MetaType.REAL: 1> <MetaType.DISCRETE: 3>]
	domains [array([-7.04209995,  6.82480001]), array([-13.7730999 ,  12.95160007]), array([-5.28609991, 17.92740059]), array([-8.54819965,  2.44950008]), array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  

       2.82485868e-10, 2.82485868e-10])}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorDirichlet object at 0x0000001EC1C4C208>}
{'params': {'p': 0.5}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorBeta object at 0x0000001EC1C4CC88>}
{'params': {'mean': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1C4C390>}
{'params': {'mean': 0, 'stdev': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorNormalInverseGamma object at 0x0000001EC1F4C9B0>}
{'params': {'alpha': 1.1, 'beta': 1}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1F4C320>}
{'params': {'l': 2}, 'prior': <bayesian.spn.algorithms.Posteriors.PriorGamma object at 0x0000001EC1F4C5C0>}




CATEGORICAL




1 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

 2.81690133e-10 2.81690133e-10])) + 0.3333333333333333*(Geometric(V4|p=0.5)) + 0.3333333333333333*(Poisson(V4|mean=2)))))) * (0.44851904090267986*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=4.213639737098829;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2))))) + 0.5514809590973202*((TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V1|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V1|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V1|l=2))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V2|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(V2|alpha=1.1;beta=1)) + 0.3333333333333333*(Exponential(V2|l=2)))))) * TypeMixtureUnconstrained(REAL, 0.3333333333333333*(Gaussian(V3|mean=0;stdev=1)) + 0.3333333333333333*(Gamma(

In [105]:
print("Accuracy of the inferred statistical types is {}".format(accuracy))

Accuracy of the inferred statistical types is 0.8


The results are summarized as follows.

In [108]:
bayesian_acc = {'Dataset_id': [31, 1464, 334, 50, 333, 1494, 3, 1510, 1489, 37,
                                   1479, 1487, 1063, 1471, 1467, 1480, 1068, 1492, 1050, 1462],
                'Accuracy': [0.57, 1, 1, 0, 1, 0.548, 0.108, 1, 0.5, 1, 0.99, 0.90, 0.955, 1, 0.905, 1 ,1, 0.98, 0.68, 0.8]}
df_bayesian_acc = pd.DataFrame(bayesian_acc) 

In [111]:
df_bayesian_acc[['Dataset_id', 'Accuracy']]

,Dataset_id,Accuracy
0,31,0.570
1,1464,1.000
2,334,1.000
3,50,0.000
4,333,1.000
5,1494,0.548
6,3,0.108
7,1510,1.000
8,1489,0.500
9,37,1.000
